# Introduction
Uses one ETF.

Model Inputs are indicators.

Multi models with dynamic training

# Results

In [1]:
import pandas as pd
from collections import Counter
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import datetime
import backtrader as bt
import sys
sys.path.insert(0, '../code')
import prepareDataV3 as util
import utils
import time
import math
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as pltNonConflict
import re
np.set_printoptions(precision=4)
import itertools

In [2]:
THRESHOLD = 0.00
TEST_PART = 0.2
IMAGE_PATH = '../output/'
IMAGE_PREFIX = '30'
IMAGE_SUFIX  = '.png'
RISK_TO_TRADE  = 0.01 # 0.01 = 1%

NUMBER_OF_RUNS = 10
DEBUG_VERBOSE = False

TRAINING_AMOUNT = 1500 # number used at previous notebooks
BIGGEST_INDICATOR_START_LOCKUP = 59
BACKTEST_START_LOCKUP = TRAINING_AMOUNT + BIGGEST_INDICATOR_START_LOCKUP
TRAINING_EXPIRE_IN = 60 # random number that goes till end of file

TESTS = [
    {
        "key": "MLP_Relu_EURUSDDaily",
        "clf": MLPClassifier(activation='relu', hidden_layer_sizes=(100)),
        "dataFileName": "EURUSDDaily",
    },
    {
        "key": "MLP_Tanh_EURUSDDaily",
        "clf": MLPClassifier(activation='tanh', hidden_layer_sizes=(100)),
        "dataFileName": "EURUSDDaily",
    },
    {
        "key": "MLP_Log_EURUSDDaily",
        "clf": MLPClassifier(activation='logistic', hidden_layer_sizes=(100)),
        "dataFileName": "EURUSDDaily",
    },
]

DEFAULT_DT_FORMAT = '%Y.%m.%d'
DEFAULT_FROM_DATE = datetime.date(2010, 7, 1)

# ENUM
NEUTRAL = 1
UP = 2
DOWN = 0

# Backtesting Strategy
Model SVM Linear (https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC)

In [3]:
class TestStrategy(bt.Strategy):
    def __init__(self):
        self.indicators = []
        self.toMean = {
            "trainScores": {"data": []},
            "testScores": {"data": []},
            "forecasts": [],
        }


        # just to set minimum_period and skip first instance
        self.sma = bt.indicators.SimpleMovingAverage(period=BACKTEST_START_LOCKUP, plot=False)

        # risk for the sizer
        self.StandardDeviation = bt.indicators.StandardDeviation(plot=False)

        self.indicators.append(bt.indicators.TrueRange(plot=False))
        self.indicators.append(bt.indicators.UltimateOscillator(plot=False))
        self.indicators.append(bt.indicators.DoubleExponentialMovingAverageOscillator(plot=False))
        self.indicators.append(bt.indicators.AverageDirectionalMovementIndex(plot=False))
        self.indicators.append(bt.indicators.HullMovingAverageOscillator(plot=False))
        self.indicators.append(bt.indicators.RSI_EMA(plot=False))
        self.indicators.append(bt.indicators.HurstExponent(plot=False))
        self.indicators.append(bt.indicators.UpMove(plot=False))
        self.indicators.append(bt.indicators.PlusDirectionalIndicator(plot=False))
        
    def trainModel(self):
        start = time.time() 
        data, target = util.prepareDataForTraining(self.datas[0], TRAINING_AMOUNT, THRESHOLD, self.indicators)
        # first model train
        self.clf.fit(data, target)
        end = time.time()
        if(DEBUG_VERBOSE):
            print(
                'Training until:', datetime.date.fromordinal(int(self.datas[0].datetime[0])),
                'Training Time:', "%.2f" % (end-start)
            )
        self.toMean['trainScores']['data'].append(self.clf.score(data, target))
        
    def checkTestScore(self):
        data, target = util.prepareDataForTraining(self.datas[0], TRAINING_EXPIRE_IN, THRESHOLD, self.indicators)
        self.toMean['testScores']['data'].append(self.clf.score(data, target))
        self.toMean['forecasts'].append(self.clf.predict(data))

    def predict(self):
        data = util.getDataFromIndicatorsForPrediction(1, self.indicators)
        prediction = self.clf.predict(data)[0]
        return prediction

    def nextstart(self):
        self.trainModel()

    def next(self):
        if(len(self) % TRAINING_EXPIRE_IN) == 0:
            self.checkTestScore()
            self.trainModel()

        prediction = self.predict()

        positionSize = self.broker.getposition(self.data).size
        if(prediction == DOWN and not positionSize < 0):
            self.sell(size=None)
        elif(prediction == UP and not positionSize > 0):
            self.buy(size=None)

    def stop(self):
        if(DEBUG_VERBOSE):
            print(self.toMean['forecasts'])
        for label, measurements in self.toMean.items():
            if 'data' not in measurements:
                continue
            if(DEBUG_VERBOSE):
                print(label, np.mean(measurements['data']))
    def notify_order(self, order):
        if(order.status in [order.Canceled, order.Margin, order.Rejected]):
            print(order)
            print(self.broker.getposition(data))


In [4]:
class MySizer(bt.Sizer):
    def _getsizing(self, comminfo, cash, data, isbuy):
        sizeNecessaryToRevert = abs(self.broker.getposition(data).size)
        tradingSize = self.getSizeOnStandardDeviation(self.strategy.StandardDeviation[0])
        return tradingSize + sizeNecessaryToRevert
        
    def getSizeOnStandardDeviation(self, stddev):
        valueToTrade = RISK_TO_TRADE * self.broker.get_value()
        return math.floor(valueToTrade/stddev)

# Backtesting

In [5]:
utils.runBackTest(TESTS, NUMBER_OF_RUNS, TestStrategy, MySizer, DEFAULT_DT_FORMAT, DEFAULT_FROM_DATE)

############################
starting MLP_Relu_EURUSDDaily
############################
end run 0
end run 1
end run 2
end run 3
end run 4
end run 5
end run 6
end run 7
end run 8
end run 9
testScore: 0.5014705882352941(+-0.012286234455414912), max: 0.526470588235294, min: 0.48235294117647065
calcMeanStd:  [0.5029411764705882, 0.4970588235294117, 0.4901960784313726, 0.48235294117647065, 0.526470588235294, 0.4980392156862745, 0.5009803921568627, 0.49607843137254903, 0.5196078431372548, 0.5009803921568627]
portfolio: 9761.921002999992(+-2426.7239293652888), max: 15439.342729999998, min: 6690.320129999967
calcMeanStd:  [9096.03806999997, 10472.80112000002, 7769.151179999977, 6870.613110000012, 10793.484910000008, 11166.043530000015, 8995.003139999984, 6690.320129999967, 15439.342729999998, 10326.41210999998]
total execution time: 144.52
############################
starting MLP_Tanh_EURUSDDaily
############################
end run 0
end run 1
end run 2
end run 3
end run 4
end run 5
end run 

In [6]:
for test in TESTS:
    # Plot the result
    figures = test['resultCerebro'].plot(style='bar', iplot=False)
    print(figures)
    figures[0][0].savefig(IMAGE_PATH + IMAGE_PREFIX + '-' + test["key"] + IMAGE_SUFIX )


[[<Figure size 1920x983 with 4 Axes>]]
[[<Figure size 1920x983 with 4 Axes>]]
[[<Figure size 1920x983 with 4 Axes>]]
